In [1]:
from pymongo import MongoClient

# Konfigurasi koneksi MongoDB
client = MongoClient('mongodb://localhost:27017/')

In [2]:
# Pilih database
db = client.inventory

# Pilih koleksi
collection = db.customers

# Simpan dokumen
result = collection.insert_one({"key": "value"})


In [5]:
from google.cloud import pubsub_v1
from google.cloud import bigquery
import json

# Inisialisasi klien Pub/Sub
subscriber = pubsub_v1.SubscriberClient()
subscription_path = 'projects/testde-foomlg/topics/tutorial.inventory.customers'

# Inisialisasi klien BigQuery
client = bigquery.Client()
dataset_id = 'test'
table_id = 'test'

# Fungsi untuk menulis data ke BigQuery
def write_to_bigquery(data):
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)
    
    # Ubah data menjadi format yang sesuai dengan skema tabel BigQuery
    row = (data['name'], data['email'], data['phone'], data['address']['street'], data['address']['city'], 
           data['address']['state'], data['address']['zipcode'], data['address']['country'], 
           json.dumps(data['orders']))

    # Masukkan data ke BigQuery
    errors = client.insert_rows(table, [row])
    if errors:
        print(f'Error inserting rows: {errors}')

# Fungsi untuk mengolah pesan dari Pub/Sub
def callback(message):
    data = json.loads(message.data)
    print(f'Received message: {data}')

    # Panggil fungsi untuk menulis ke BigQuery
    write_to_bigquery(data)

    # Acknowledge message
    message.ack()

# Subscribe ke topik Pub/Sub
subscriber.subscribe(subscription_path, callback=callback)

# Biarkan program berjalan terus
print('Listening for messages on {}'.format(subscription_path))
try:
    while True:
        pass
except KeyboardInterrupt:
    print('Stopped listening.')


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [4]:
pip install google-cloud-pubsub google-cloud-bigquery

   ---------------------------------------- 0.0/273.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/273.2 kB ? eta -:--:--
   ----- --------------------------------- 41.0/273.2 kB 653.6 kB/s eta 0:00:01
   ----------------------- ---------------- 163.8/273.2 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 273.2/273.2 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\metranet\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from google.cloud import pubsub_v1
import json

project_id = "testde-foomlg"
subscription_id = "tutorial.inventory.customers"

# Data oplog yang telah diambil sebelumnya
oplog_data = {
    1717650545: 1717650545000,  # Contoh timestamp dan ts_ms
    1717650565: 1717650565000,
    # Tambahkan data oplog lainnya sesuai kebutuhan
}

def callback(message):
    data = json.loads(message.data.decode("utf-8"))
    if data["op"] == "r" and data["source"]["ts_ms"] == 0:
        oplog_ts = data["source"]["ts"]  # Mengambil timestamp dari source
        if oplog_ts in oplog_data:
            data["source"]["ts_ms"] = oplog_data[oplog_ts]
            print(f"Updated ts_ms: {data['source']['ts_ms']} for oplog_ts: {oplog_ts}")
    # Process modified message
    print(data)
    message.ack()

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_id)
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

with subscriber:
    try:
        streaming_pull_future.result()
    except KeyboardInterrupt:
        streaming_pull_future.cancel()
        streaming_pull_future.result()


ModuleNotFoundError: No module named 'google'